# Assinatura digital
Nas suposições feitas até então sobre criptografia, criamos duas pessoas teóricas, nossos amigos Alice e Bob, que trocavam mensagens secretas um com o outro. Alice e Bob não queriam que Eva, uma terceira pessoa, lesse as mensagens. Os métodos de criptografia que vimos (Afim, exponencial e RSA), se provam bastante efetivos, devido à dificuldade de se calcular ou fatorar números primos.

Todavia, o que acontece se Eva for extremamente inteligente e conseguir decifrar a criptografia? Como Bob pode _comprovar_ que o remetente era, de fato, Alice? Isso é feito por meio da Assinatura digital.

## Como Implementar uma Assinatura digital?
1. Alice e Bob irão escolher suas funções de Codificação ($C_a\ e\ C_b$) e Decodificação ($D_a\ e\ D_b$).

2. Alice quer enviar uma mensagem para Bob (como sempre). Para isso, ela irá:
    1. Criptografar a mensagem $b$ calculando $c = C_b(b) $
    2. Anexar a "Assinatura" $a = D_a(b)$.
3. Bob, recebendo as mensagens $c$ e $a$, fará o seguinte:
    1. Descriptografará $c$ usando sua chave de decriptografia ($b = D_b(c)$)
    2. Aplicará a função de codificação de Alice ($C_a$) na assinatura $a$, e verifica se $b = C_a(a) = D_b(c)$

Se a igualdade acima for verdadeira, quer dizer que a mensagem realmente foi enviada pela Alice, mantendo a confiabilidade do canal.

### Exemplo de assinatura digital
Supondo que mantemos os dados do exemplo anterior (Notebook de criptografia RSA), com a chave pública de Bob sendo (53, 53531),
e a chave privada sendo (24029, 53064). Precisaremos escolher uma chave pública e privada para Alice também.

In [1]:
def codifica_bloco(mesg:str, size=20) -> list:
  """Função para codificar uma mensagem de acordo com o tamanho do Bloco"""
  
  bloco, n =  0, 0
  lista = []
  l_final = []
  
  while(n + size <= len(mesg)):
    lista.append(mesg[n:n + size])
    n += size
    
  if len(mesg)%size != 0:
    blocos_criados = (len(mesg) // size)
    lista.append(mesg[(blocos_criados * size):])

  for i in lista:
    mes = i
    
    for idx, s in enumerate(mes):
      bloco += ord(s) * 256**idx 
      
    l_final.append(bloco)
    bloco = 0
    
  return l_final


def decodifica_bloco(num):
  """Função para descodificar uma mensagem em blocos."""
  b = []

  for i in num:
    a = []
    c = []
    
    while i > 0:
      a.append(i%256)
      i = i//256

    
    for j in a:
      c.append(chr(j))
    b.append(str(c).strip().replace(',', '').replace('[', '').replace(']', ' ').replace("'", ''))
    
  return b

# Aqui serão definidas funções

def XMDC(a, b):
  """Algoritmo Extendido de Euclides, para cálculo das equações modulares"""
  x0, x = 1, 0
  y0, y = 0, 1
  
  sign_a, sign_b = 1,  1
  if a < 0:
    a = -a
    sign_a = -1
  
  if b < 0:
    b = -b
    sign_b = -1
    
  while b > 0:
    q, r = a//b, a%b
    a, b = b, r
    
    x, x0 = x0 - q * x, x
    y, y0 = y0 - q * y, y
    
  return a, sign_a * x0, sign_b * y0

def ExpModN(a, e, n):
  """Calcula a^e mod n"""
  A, P, E = a, 1, e
  while E != 0:
    D = E % 2
    
    if D == 1:
      P = (A*P) % n
      E = (E-1)//2
      
    else:
      E = E//2
      
    A = (A*A) % n
    
  return P

def inverso_modular(a, n):
  """Calcula o inverso modular de a em Zn"""
  d, u, v = XMDC(a, n)
  
  if d !=1: 
    return False
  
  u %= n
  
  if u < 0: 
    u += n
    
  return u

criptografa = lambda b, e, n: (b**e) % n
descriptografa = lambda c, d, n: (c**d) % n 

In [30]:
#Definindo aqui os primos e chaves para ambos os indivíduos
p_b, q_b = 269, 199
p_a, q_a = 311, 263

# Calculando n
n_a = p_a*q_a
n_b = p_b*q_b

# Calculando Phi_n
phi_a = (p_a - 1)*(q_a - 1)
phi_b = (p_b - 1)*(q_b - 1)

# Definindo valor e | mdc(e, phi) = 1
e_a = 53
e_b = 53

# Definindo o valor 
d_a = inverso_modular(e_a, phi_a)
d_b = inverso_modular(e_b, phi_b)

# Chaves
c_publica_a, c_privada_a = (n_a, e_a), (phi_a, d_a)
c_publica_b, c_privada_b = (n_b, e_b), (phi_b, d_b)
 

In [31]:
# Aqui teremos alice criptografando e assinando uma mensagem.
mensagem = "Oi"

# Codificando mensagem
mensagem_cod = codifica_bloco(mensagem, 6)[0]

#Criptografando Mensagem
mensagem_cripto = criptografa(mensagem_cod, e_b, n_b)

# Gerando assinatura
assinatura = descriptografa(mensagem_cod, d_a, n_a)

mensagem_alice = (mensagem_cripto, assinatura)
print(f"""Mensagem Codificada: {mensagem_cod}\nMensagem Criptografada: {mensagem_cripto}\nAssinatura: {assinatura}""")

Mensagem Codificada: 26959
Mensagem Criptografada: 37220
Assinatura: 14907


In [35]:
# Agora, Bob, em posse da mensagem assinada, irá descriptografar

# Desempacotando
mensagem, assinatura = mensagem_alice

# Descriptografando
mensagem_decrip = descriptografa(mensagem, d_b, n_b)

# Decodificando Assinatura
assinatura = criptografa(assinatura, e_a, n_a)

# Decodificando Mensagem se for igual à assinatura:
if assinatura == mensagem_decrip:
  m = decodifica_bloco([mensagem_decrip])
  print(m)
  
# Por fim, verificamos:
print(f"""Mensagem Descriptografada: {mensagem_decrip}\nAssinatura Decod: {assinatura}\nAssinatura é igual à Mensagem? {assinatura == mensagem_decrip}""")

['O i ']
Mensagem Descriptografada: 26959
Assinatura Decod: 26959
Assinatura é igual à Mensagem? True


## Troca de Chaves Diffie-Hellman

Uma outra situaçao possível é Alice e Bob quererem utilizar uma chave em comum para trabalhar a troca de mensagens, pelo motivo que for. Todavia, como já vimos, se Bob enviar a chave privada para Alice, Eva poderá ver tal chave e, eventualmente, decodificar mensagens secretas entre os dois. É necessário um meio de enviar a chave por um canal aberto, portanto, sem que esta possa ser lida. O procedimento de Diffie-Hellman soluciona este problema, e é baseado no fato de não conhecermos solução eficiente para o problema do _Logaritmo Discreto_, enunciado à seguir:
* Seja $p$ um primo (notadamente grande) e $\=g$ um elemento primitivo de $\mathbb{Z_p}$. (Elemento primitivo é aquele que obedece $\left\{\=g^0, ... \=g^{p-2} \right\} \equiv \left\{1, ..., p-1 \right\} \in \mathbb Z_{p}$). Dado $\=g^k$, determine $k$.

### O procedimento Diffie-Hellman

1. Alice e Bob escolhem um primo $p$ grande e um elemento primitivo $\=g \in \mathbb Z_p$.
2. Alice escolhe um inteiro secreto $a \in \left\{1,...,p-1 \right\}$ e envia $\=x = \=g^a$ ao Bob.
3. Bob escolhe um inteiro secreto $b \in \left\{1,...,p-1\right\}$ e envia $\=y = \=g^b$ à Alice.
Com isso, ambos conseguem um valor idêntico ao realizar:
$$
\=y^a = (\=g^b)^a = \=g^{ab} = (\=g^a)^b = \=x^b
$$
O valor $\=g^{ab}$ é a chave secreta mutuamente conhecida.

#### Exemplo
Vamos agora exemplificar o funcionamento da criptografia com um pedaço de código:
(Note que para QUALQUER valor de $a,\ b$, com $a, b < p$, a chave continua sendo a mesma.)

In [42]:
# Supondo o primo escolhido por ambos p = 23 e o primitivo g = 5
p = 23
g = 5

# Alice escolhe a = 10 e Bob b = 20 
## ALTERE a E b CONFORME QUISER, SEMPRE MENOR QUE P. ##

a = 11
b = 23
x = 5**a % 23
y = 5**b % 23

print(f"Valor de x: {x}, Valor de y: {y}")

# Finalmente:
chave_a = y**a % 23
chave_b = x**b % 23

print(f"Chaves são iguais? {chave_a == chave_b}\nValor da Chave: {chave_a}")

Valor de x: 22, Valor de y: 5
Chaves são iguais? True
Valor da Chave: 22
